<a href="https://colab.research.google.com/github/Lutris98/21_1Q/blob/master/5.House/House_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.EDA

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor,VotingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [10]:
from google.colab import drive
drive.mount('/content/gdrive')
train_df=pd.read_csv('/content/gdrive/MyDrive/Dataset/house_train.csv')
test_df=pd.read_csv('/content/gdrive/MyDrive/Dataset/house_test.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


##1)Basic statistics

##2)Basic feature selection


##3)Encoding

#2.Preprocessing(with Visualization)

##1)Null filling

##2)Scaling&Outlier removal

##3)feature selection

#3.Modeling

#1)Uploading

#2)Feedback
baseline : 

#3)Afterwords
regression problems are mostly time series analysis anyways